<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-2%20%EA%B5%90%EC%B0%A8%20%EA%B2%80%EC%A6%9D%EA%B3%BC%20%EA%B7%B8%EB%A6%AC%EB%93%9C%20%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

## 검증 세트

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

In [2]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
data = wine.iloc[:, :-1]
target = wine.iloc[:, -1]

In [4]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

훈련세트가 2만이 넘는다면 훈련세트로만 전체를 사용하고 val set는 없어도 되고, 데이터가 충분히 많다면 val set는 5~ 10%만 지정해서 사용해도 됨

In [6]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [7]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

DecisionTreeClassifier(random_state=42)

In [8]:
sub_score = dt.score(sub_input, sub_target)
val_score = dt.score(val_input, val_target)
print('sub_score : {} \nval_score : {}'.format(sub_score, val_score))

sub_score : 0.9971133028626413 
val_score : 0.864423076923077


## 교차 검증 <br>
ML에서는 샘플수가 충분히 많지 않기 때문에 교차검증을 시행하여 최적의 파마미터을 찾게 되지만, 딥러닝에서는 샘플 수가 충분하여 교차검증을 하지 않고, 훈련세트와 검증세트로만 나누어 사용함 <br>
따라서, tensorflow, keras같은 경우는 gridsearchcv와 같은 기능이 그렇게 많지 않음. tensorflow, keras는 데이터을 읽어들이는 도구들이 많이 발달되어 있음.

In [9]:
from sklearn.model_selection import cross_validate

In [10]:
dt = DecisionTreeClassifier(random_state=42)

In [11]:
scores = cross_validate(dt,
                        X = train_input, y=train_target,
                        cv=5, n_jobs=multiprocessing.cpu_count(),
                        verbose=1)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    4.3s finished


In [12]:
scores

{'fit_time': array([0.02489161, 0.01871085, 0.04998875, 0.06989479, 0.04695678]),
 'score_time': array([0.00915241, 0.01088166, 0.00975943, 0.01608396, 0.00424838]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [13]:
print("test score : {} (+/- {}".format(scores['test_score'].mean(), scores['test_score'].std()))

test score : 0.855300214703487 (+/- 0.015339373270258668


'# dt.score(test_input, test_target) <br>
cross_validate에서의 fit기능은 무엇인가? <br>
.fit는 언제하는가?

#### 분류기(splitter) 사용 <br>
클라스가 고르게 섞이게 하기 위하여 사용 <br>
회귀모델에서는 KFold() 객체, 분류모델에서는 StratifiedKFold() 사용 <br>
cross_validate method에서는 estimator에 따라 분류인지, 회귀인지를 확인하고, cv를 StratifiedKFlod()를 사용할지 KFold를 사용할 지 자동으로 결정하여 진행함.

In [14]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt,
                        X = train_input, y=train_target,
                        cv=StratifiedKFold(5), 
                        n_jobs=multiprocessing.cpu_count(),
                        verbose=1)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.2s finished


In [15]:
print("test score : {} (+/- {}".format(scores['test_score'].mean(), scores['test_score'].std()))

test score : 0.855300214703487 (+/- 0.015339373270258668


위에서 cv=5를 적용한 cross_validate결과와 동일한 결과가 나옴.<br>
아래 splitter를 사용하여 상세하게 파라미터를 구성하여 사용

In [16]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print("test score : {} (+/- {}".format(scores['test_score'].mean(), scores['test_score'].std()))

test score : 0.8574181117533719 (+/- 0.012294268284775105


## 하이퍼파라미터 튜닝 <br>
최적의 파라미터를 찾기 위한 loop실행

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [19]:
# dt = DecisionTreeClassifier(random_state=42)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                  params,
                  n_jobs=-1) # -1:모든 cpu core를 사용
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [20]:
dt = gs.best_estimator_
dt_score = dt.score(train_input, train_target)
print('best estimator score: {}'.format(dt_score))

best estimator score: 0.9615162593804117


In [21]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [22]:
for k, v in gs.cv_results_.items():
  if k == 'params':
    print('params')
    for k, v in params.items():
      print(k, v)
    continue
  print('{}: {}'.format(k, v))

mean_fit_time: [0.02222834 0.01694598 0.03477678 0.04159598 0.03046489]
std_fit_time: [0.00586899 0.00676781 0.01236292 0.00745372 0.00929594]
mean_score_time: [0.00760889 0.01297731 0.01768322 0.0103476  0.00835552]
std_score_time: [0.00350503 0.01198415 0.00294716 0.00624305 0.00692635]
param_min_impurity_decrease: [0.0001 0.0002 0.0003 0.0004 0.0005]
params
min_impurity_decrease [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
split0_test_score: [0.86923077 0.87115385 0.86923077 0.86923077 0.86538462]
split1_test_score: [0.86826923 0.86346154 0.85961538 0.86346154 0.86923077]
split2_test_score: [0.8825794  0.87680462 0.87584216 0.88161694 0.8825794 ]
split3_test_score: [0.86717998 0.85466795 0.85081809 0.84889317 0.84985563]
split4_test_score: [0.85370549 0.85659288 0.86910491 0.87584216 0.87102984]
mean_test_score: [0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
std_test_score: [0.00915386 0.00843731 0.0087452  0.01125985 0.01056953]
rank_test_score: [1 5 4 2 3]


In [23]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [24]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [25]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [26]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [27]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

In [28]:
from scipy.stats import uniform, randint
# from numpy.random import uniform, randint # 기능은 같고 모양은 틀리나, 객체를 생성하지 못함.(아래 예 참조)

In [29]:
rgen = randint(0, 10)
rgen.rvs(10)

array([5, 1, 8, 9, 7, 9, 4, 9, 8, 5])

In [30]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([103,  98,  96, 101, 106,  93, 116,  98,  97,  92]))

In [31]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.28315498, 0.92691237, 0.91520592, 0.29229064, 0.11947462,
       0.1751068 , 0.1733081 , 0.41181769, 0.97621625, 0.61736035])

In [32]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [33]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42) #n_iter: params 의 임의변수를 100번 시행
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa17bd2d290>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa17bd2da50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa17c648b90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa17bd2db50>},
                   random_state=42)

In [34]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [35]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [36]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [37]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa17bd2d290>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa17bd2da50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa17c648b90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa17bd2db50>},
                   random_state=42)

In [38]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
